In [1]:
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
import random
from zipfile import ZipFile
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog
from skimage.io import imread
from skimage import color, exposure
from skimage.transform import rescale
from skimage.util import view_as_blocks
from multiprocessing import Pool
from skimage.color import rgb2gray
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import multiprocess as mp
import pickle
import tensorflow as tf
from multiprocessingNotebook import runt

PICKLE_PATH = 'test_data.pickle'

NUM_TRAINING_IMAGES = 500
NUM_TESTING_IMAGES = 50

NUM_SPACES_PER_BOARD = 2
PICTURE_DIMENSIONS = 400
block_width = PICTURE_DIMENSIONS // 8
block_height = PICTURE_DIMENSIONS // 8
num_channels = 3

2024-05-06 12:49:45.169876: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 12:49:46.897493: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from zipfile import ZipFile

def getFilePaths(filePath):
    # opens the chess boards.zip file into the chessBoards object
    chessBoards = ZipFile(filePath, 'r')

    test = []
    train = []

    # extract the file names from the train and test folders in the zip archive
    for file in chessBoards.namelist():
        # fills the file names from within the dataset subfolder
        if file[:4] == 'test':
            test.append(file)
        if file[:5] == 'train':
            train.append(file)

    return train, test

zipFilePath = "../chess boards.zip"

trainPaths, testPaths = getFilePaths(zipFilePath)
print("Files in test: ", len(trainPaths))
print("Files in train: ", len(testPaths))

Files in test:  80000
Files in train:  20000


In [3]:
# Parallel Loads Board Images for Training Purposes
def HogTransform(img):
    first_image_gray = color.rgb2gray(img)

    fd, hog_image = hog(
        first_image_gray,
        orientations=8,
        pixels_per_cell=(8, 8),
        cells_per_block=(1, 1),
        visualize=True,
        block_norm='L2-Hys',
        feature_vector=True
    )

    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    hog_image_uint8 = (hog_image_rescaled * 255).astype(np.uint8)
    return hog_image_uint8

def read_and_store_modified_chess_images():
    return

def fen_from_position(position, fen_string):
    rows = position // 8
    cols = position % 8

    character = fen_string[rows * 8 + cols]

    if character.isdigit():
        return ' ' * int(character)
    else:
        return character

def fen_from_filename(filename):
    parts = filename.split('/')
    fen_part = parts[-1].split('.')[0]

    fen_string = ''.join([' ' * int(char) if char.isdigit() else char for char in fen_part])
    fen_string = fen_string.replace('-', '')

    return fen_string

src = zipFilePath
random.shuffle(trainPaths)

print("Running...")
lock = mp.Lock()
manager = mp.Manager()
processorCores = 12
#data_dict = manager.dict({'fenstring': [], 'data': []})
data_dict = manager.list()
with mp.Pool(processes=processorCores) as p:
    args = [(fileName, src, fen_from_filename, fen_from_position, NUM_SPACES_PER_BOARD, data_dict) for fileName in trainPaths[:NUM_TRAINING_IMAGES]]
    results = [0]
    results = p.starmap(runt, args)

# Access the shared dictionary
print(len(data_dict))

combined_results = {'fenstring': [], 'data': []}

# Initialize lists to store all data and fenstring items
all_data_items = []
all_fenstring_items = []

for result in data_dict:
    # Extend all_data_items with result['data']
    all_data_items.extend(result['data'])

    # Extend all_fenstring_items with result['fenstring']
    all_fenstring_items.extend(result['fenstring'])

# Assign the combined lists to combined_results
combined_results['data'] = all_data_items
combined_results['fenstring'] = all_fenstring_items

joblib.dump(combined_results, PICKLE_PATH)
read_and_store_modified_chess_images()

Running...
1890


In [4]:
print(tf.__version__)

# shows a list of gpu devices recognized by tensorflow
#print(tf.config.list_physical_devices('GPU'))


import os
"""
0 = all messages are logged (default behavior)
1 = INFO messages are not printed
2 = INFO and WARNING messages are not printed
3 = INFO, WARNING, and ERROR messages are not printed """
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

# download fashion mnist dataset and split into train and test sets
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# create a list of the labels from the data
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# shows the number of images and their resolution and num of training labels
print(train_images.shape)
print(len(train_labels))

# scale images between 0 to 1
train_images = train_images / 255.0
test_images = test_images / 255.0


# show the first 25 images and their labels in training set
#plt.figure(figsize=(10,10))
#for i in range(25):
    #plt.subplot(5,5,i+1)
    #plt.xticks([])
    #plt.yticks([])
    #plt.grid(False)
    #plt.imshow(train_images[i], cmap=plt.cm.binary)
    #plt.xlabel(class_names[train_labels[i]])
#plt.show()

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(28,28)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(784, activation='relu'))
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.Dense(10))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10)

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

2.16.1
(60000, 28, 28)
60000


2024-05-06 12:54:51.802143: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 12:54:52.063612: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 12:54:52.063764: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 12:54:52.066008: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 12:54:52.066139: I external/local_xla/xla/stream_executor

Epoch 1/10


I0000 00:00:1715014494.805857    6708 service.cc:145] XLA service 0x7f4538005c70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715014494.805980    6708 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 with Max-Q Design, Compute Capability 7.5
2024-05-06 12:54:54.874381: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-06 12:54:55.138928: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


  42/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.4772 - loss: 1.4314

I0000 00:00:1715014496.309342    6708 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7852 - loss: 0.5951
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8640 - loss: 0.3718
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8760 - loss: 0.3332
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8868 - loss: 0.3072
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8923 - loss: 0.2870
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9003 - loss: 0.2685
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9026 - loss: 0.2566
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9082 - loss: 0.2424
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9105 - loss: 0.2346
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9148 - loss: 0.2262
313/313 - 2s - 5ms/step - accuracy: 0.8830 - loss: 0.3417

Test accuracy: 0.8830000162124634
